In [1]:
import os 
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import pandas as pd,requests,bs4,re,time,io,pytesseract,easyocr,random,textstat,urllib.request
from pdfminer.high_level import extract_text
from PIL import Image
from pathlib import Path
from pdf2image import convert_from_path
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from goose3 import Goose
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm.auto import tqdm

reader = easyocr.Reader(['en'])

import warnings
warnings.filterwarnings("ignore")

%autosave 1

Autosaving every 1 seconds


In [2]:
df = pd.read_csv("NHSinform disease_data.csv")
df

,disease,text,url
0,Abdominal aortic aneurysm,About abdominal aortic aneurysms\nAn abdominal...,https://www.nhsinform.scot/illnesses-and-condi...
1,Acne,About acne\nAcne is a common skin condition th...,https://www.nhsinform.scot/illnesses-and-condi...
2,Acute cholecystitis,About acute cholecystitis\nAcute cholecystitis...,https://www.nhsinform.scot/illnesses-and-condi...
3,Acute lymphoblastic leukaemia,About acute lymphoblastic leukaemia\nLeukaemia...,https://www.nhsinform.scot/illnesses-and-condi...
4,Acute lymphoblastic leukaemia: Children,Introduction\nAcute lymphoblastic leukaemia (A...,https://www.nhsinform.scot/illnesses-and-condi...
...,...,...,...
317,Warts and verrucas,About warts and verrucas\nWarts are small lump...,https://www.nhsinform.scot/illnesses-and-condi...
318,Whooping cough,"About whooping cough\nWhooping cough, also cal...",https://www.nhsinform.scot/illnesses-and-condi...
319,Wilms’ tumour,Introduction\nWilms’ tumour is a type of kidne...,https://www.nhsinform.scot/illnesses-and-condi...
320,Womb (uterus) cancer,About womb cancer\nCancer of the womb (uterus)...,https://www.nhsinform.scot/illnesses-and-condi...


In [3]:
df.isna().sum()

disease     0
text       17
url         0
dtype: int64

In [4]:
df_na = df[df['text'].isna()]
df_na.reset_index(drop=True,inplace=True)
df_na

,disease,text,url
0,Chronic fatigue syndrome,NaN,https://www.nhsinform.scot/illnesses-and-condi...
1,Coronavirus (COVID-19),NaN,https://www.nhsinform.scot/illnesses-and-condi...
2,Dementia,NaN,https://www.nhsinform.scot/illnesses-and-condi...
3,Diabetes,NaN,https://www.nhsinform.scot/illnesses-and-condi...
4,Foetal alcohol syndrome,NaN,https://www.nhsinform.scot/illnesses-and-condi...
5,Langerhans cell histiocytosis,NaN,https://www.nhsinform.scot/illnesses-and-condi...
6,Malnutrition,NaN,https://www.nhsinform.scot/illnesses-and-condi...
7,Menopause,NaN,https://www.nhsinform.scot/illnesses-and-condi...
8,Norovirus,NaN,https://www.nhsinform.scot/illnesses-and-condi...
9,Pregnancy and baby,NaN,https://www.nhsinform.scot/illnesses-and-condi...


In [5]:
import webbrowser

#this will open all the link to verify manually
for x in range(len(df_na)):    
#     webbrowser.open(df_na.iloc[x]['url']) 
    print(df_na.iloc[x]['disease'],df_na.iloc[x]['url'])

Chronic fatigue syndrome https://www.nhsinform.scot/illnesses-and-conditions/a-to-z/c/chronic-fatigue-syndrome
Coronavirus (COVID-19) https://www.nhsinform.scot/illnesses-and-conditions/a-to-z/c/coronavirus-covid-19
Dementia https://www.nhsinform.scot/illnesses-and-conditions/a-to-z/d/dementia
Diabetes https://www.nhsinform.scot/illnesses-and-conditions/a-to-z/d/diabetes
Foetal alcohol syndrome https://www.nhsinform.scot/illnesses-and-conditions/a-to-z/f/foetal-alcohol-syndrome
Langerhans cell histiocytosis https://www.nhsinform.scot/illnesses-and-conditions/a-to-z/l/langerhans-cell-histiocytosis
Malnutrition https://www.nhsinform.scot/illnesses-and-conditions/a-to-z/m/malnutrition
Menopause https://www.nhsinform.scot/illnesses-and-conditions/a-to-z/m/menopause
Norovirus https://www.nhsinform.scot/illnesses-and-conditions/a-to-z/n/norovirus
Pregnancy and baby https://www.nhsinform.scot/illnesses-and-conditions/a-to-z/p/pregnancy-and-baby
Stress, anxiety and low mood https://www.nhsinfo

In [6]:
#driver for operation
from webdriver_manager.chrome import ChromeDriverManager
option = webdriver.ChromeOptions()
# option.add_argument('headless')
driver = webdriver.Chrome(ChromeDriverManager().install(),options=option)



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [C:\Users\AKSHAY SATPUTE\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


In [7]:
df_na1 = df_na[df_na['disease'].isin(["Foetal alcohol syndrome","Sudden infant death syndrome (SIDS)"])]
df_na1.reset_index(drop=True,inplace=True)
df_na1

,disease,text,url
0,Foetal alcohol syndrome,NaN,https://www.nhsinform.scot/illnesses-and-condi...
1,Sudden infant death syndrome (SIDS),NaN,https://www.nhsinform.scot/illnesses-and-condi...


In [8]:
for x in tqdm(range(len(df_na1))):    
    link = df_na1.iloc[x]['url']
    driver.get(link)
    
    #cookies
    try:
        driver.find_element_by_xpath('''//*[@id="ccc-notify-accept"]''').click()
    except:
        pass

    text = driver.find_element(By.XPATH,f'''//*[@id="maincontent"]/div[4]/div[2]/div[1]/div[1]/div[2]''').text.strip()

    df_na1.iloc[x]['text'] = text

  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
df_na1

,disease,text,url
0,Foetal alcohol syndrome,There's no known safe limit of drinking during...,https://www.nhsinform.scot/illnesses-and-condi...
1,Sudden infant death syndrome (SIDS),For the first 6 months of your baby's life the...,https://www.nhsinform.scot/illnesses-and-condi...


In [10]:
for i in tqdm(range(len(df_na1))):
    ii = (df[df['disease'] == df_na1.iloc[i]['disease']]).index[0]
    df.iloc[ii]['text'] = df_na1.iloc[i]['text']

  0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
df[df['disease'].isin(["Foetal alcohol syndrome","Sudden infant death syndrome (SIDS)"])]

,disease,text,url
117,Foetal alcohol syndrome,There's no known safe limit of drinking during...,https://www.nhsinform.scot/illnesses-and-condi...
279,Sudden infant death syndrome (SIDS),For the first 6 months of your baby's life the...,https://www.nhsinform.scot/illnesses-and-condi...


In [12]:
df.isna().sum()

disease     0
text       15
url         0
dtype: int64

In [13]:
df.dropna(inplace=True)

In [14]:
df.isna().sum()

disease    0
text       0
url        0
dtype: int64

In [15]:
links,diseases = [],[]
for i in range(len(df)):
    if textstat.lexicon_count(df.iloc[i]['text'], removepunct=True) < 10:
        print(df.iloc[i]['disease'])
        print(df.iloc[i]['url'])
        print(df.iloc[i]['text'])
        links.append(df.iloc[i]['url'])
        diseases.append(df.iloc[i]['disease'])
        print()

Chest pain
https://www.nhsinform.scot/illnesses-and-conditions/a-to-z/c/chest-pain
Last updated:
06 April 2022

Chilblains
https://www.nhsinform.scot/illnesses-and-conditions/a-to-z/c/chilblains
Last updated:
09 February 2022

Chlamydia
https://www.nhsinform.scot/illnesses-and-conditions/a-to-z/c/chlamydia
Last updated:
31 January 2022

Common heart conditions
https://www.nhsinform.scot/illnesses-and-conditions/a-to-z/c/common-heart-conditions
Last updated:
13 February 2020

Cough
https://www.nhsinform.scot/illnesses-and-conditions/a-to-z/c/cough
Last updated:
31 January 2022

Dermatitis herpetiformis
https://www.nhsinform.scot/illnesses-and-conditions/a-to-z/d/dermatitis-herpetiformis
Last updated:
09 February 2022

Dizziness (Lightheadedness)
https://www.nhsinform.scot/illnesses-and-conditions/a-to-z/d/dizziness-lightheadedness
Last updated:
19 October 2021

Ectopic pregnancy
https://www.nhsinform.scot/illnesses-and-conditions/a-to-z/e/ectopic-pregnancy
How can we improve this page?


In [16]:
texts = []

for link in tqdm(links[:]):
    
    retry_count = 0
    while retry_count < 3:
        try:
            driver.get(link)
            #cookies
            try:
                driver.find_element_by_xpath('''//*[@id="ccc-notify-accept"]''').click()
            except:
                pass

            text = ""
            try:
                for x in range(100):
                    ps = driver.find_element(By.XPATH,f'''//*[@id="maincontent"]/div[2]/div/div[2]/div[1]/div[2]''').text.strip()
                    text += ps
                    driver.find_element_by_xpath('''//a[@class="js-guide__incremental--previous nav-incremental-link page-navigation__next"]''').click()
            except:
                pass

            if textstat.lexicon_count(text, removepunct=True) < 10:
                print(link)
                print(text)
                retry_count += 1
            else:
                retry_count = 3
        except:
            pass

    texts.append(text)

  0%|          | 0/25 [00:00<?, ?it/s]

In [17]:
#zippig all the data togather 
zipped = list(zip(diseases,texts,links))

In [18]:
df_na2 = pd.DataFrame(zipped,columns=['disease','text','url'])
df_na2

,disease,text,url
0,Chest pain,Immediate action required:\nPhone 999 immediat...,https://www.nhsinform.scot/illnesses-and-condi...
1,Chilblains,"Chilblains are small, itchy swellings on the s...",https://www.nhsinform.scot/illnesses-and-condi...
2,Chlamydia,Chlamydia is a bacterial infection. It's one o...,https://www.nhsinform.scot/illnesses-and-condi...
3,Common heart conditions,There are many different heart conditions and ...,https://www.nhsinform.scot/illnesses-and-condi...
4,Cough,A cough is a reflex action to clear your airwa...,https://www.nhsinform.scot/illnesses-and-condi...
5,Dermatitis herpetiformis,Dermatitis herpetiformis (DH) is an autoimmune...,https://www.nhsinform.scot/illnesses-and-condi...
6,Dizziness (Lightheadedness),Dizziness (lightheadedness) is a common sympto...,https://www.nhsinform.scot/illnesses-and-condi...
7,Ectopic pregnancy,An ectopic pregnancy is when a fertilised egg ...,https://www.nhsinform.scot/illnesses-and-condi...
8,Endometriosis,Endometriosis is a long-term (chronic) conditi...,https://www.nhsinform.scot/illnesses-and-condi...
9,Fever in adults,Coronavirus (COVID-19)\nIf you're worried abou...,https://www.nhsinform.scot/illnesses-and-condi...


In [19]:
df.reset_index(drop=True,inplace=True)

In [20]:
df

,disease,text,url
0,Abdominal aortic aneurysm,About abdominal aortic aneurysms\nAn abdominal...,https://www.nhsinform.scot/illnesses-and-condi...
1,Acne,About acne\nAcne is a common skin condition th...,https://www.nhsinform.scot/illnesses-and-condi...
2,Acute cholecystitis,About acute cholecystitis\nAcute cholecystitis...,https://www.nhsinform.scot/illnesses-and-condi...
3,Acute lymphoblastic leukaemia,About acute lymphoblastic leukaemia\nLeukaemia...,https://www.nhsinform.scot/illnesses-and-condi...
4,Acute lymphoblastic leukaemia: Children,Introduction\nAcute lymphoblastic leukaemia (A...,https://www.nhsinform.scot/illnesses-and-condi...
...,...,...,...
302,Warts and verrucas,About warts and verrucas\nWarts are small lump...,https://www.nhsinform.scot/illnesses-and-condi...
303,Whooping cough,"About whooping cough\nWhooping cough, also cal...",https://www.nhsinform.scot/illnesses-and-condi...
304,Wilms’ tumour,Introduction\nWilms’ tumour is a type of kidne...,https://www.nhsinform.scot/illnesses-and-condi...
305,Womb (uterus) cancer,About womb cancer\nCancer of the womb (uterus)...,https://www.nhsinform.scot/illnesses-and-condi...


In [21]:
for i in tqdm(range(len(df_na2))): 
    ii = df[df['url'] == df_na2.iloc[i]['url']].index[0]
    df.iloc[ii]['text'] = df_na2.iloc[i]['text'] 

  0%|          | 0/25 [00:00<?, ?it/s]

In [22]:
for i in range(len(df)):
    if textstat.lexicon_count(df.iloc[i]['text'], removepunct=True) < 10:
        print(df.iloc[i]['disease'])
        print(df.iloc[i]['url'])
        print(df.iloc[i]['text'])
        print()

In [23]:
df

,disease,text,url
0,Abdominal aortic aneurysm,About abdominal aortic aneurysms\nAn abdominal...,https://www.nhsinform.scot/illnesses-and-condi...
1,Acne,About acne\nAcne is a common skin condition th...,https://www.nhsinform.scot/illnesses-and-condi...
2,Acute cholecystitis,About acute cholecystitis\nAcute cholecystitis...,https://www.nhsinform.scot/illnesses-and-condi...
3,Acute lymphoblastic leukaemia,About acute lymphoblastic leukaemia\nLeukaemia...,https://www.nhsinform.scot/illnesses-and-condi...
4,Acute lymphoblastic leukaemia: Children,Introduction\nAcute lymphoblastic leukaemia (A...,https://www.nhsinform.scot/illnesses-and-condi...
...,...,...,...
302,Warts and verrucas,About warts and verrucas\nWarts are small lump...,https://www.nhsinform.scot/illnesses-and-condi...
303,Whooping cough,"About whooping cough\nWhooping cough, also cal...",https://www.nhsinform.scot/illnesses-and-condi...
304,Wilms’ tumour,Introduction\nWilms’ tumour is a type of kidne...,https://www.nhsinform.scot/illnesses-and-condi...
305,Womb (uterus) cancer,About womb cancer\nCancer of the womb (uterus)...,https://www.nhsinform.scot/illnesses-and-condi...


In [24]:
df.to_csv("NHSinform disease_data_missing_dealt.csv",index=False)

In [25]:
import contextlib

#data to text files
for x in tqdm(range(len(df))):
    disease = df.iloc[x]['disease']
    text = df.iloc[x]['text']
    
    #saving the file to desired location 
    file_path = f'C:/Users/AKSHAY SATPUTE/Jupyter notebook/web scraping/nhsinform-a-z-disease-data/text_files/{disease}.txt'
    
    with open(file_path, "w") as o:
        with contextlib.redirect_stdout(o):
            print(f"{disease}\n\n{text}")

  0%|          | 0/307 [00:00<?, ?it/s]

In [29]:
def remove_esc_chars(string):
    review = re.sub('''[^a-zA-Z0-9!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~]''', ' ',string)
    review = review.split()
    review = ' '.join(review)
    return review

In [30]:
import pandas as pd
df = pd.read_csv("NHSinform disease_data_missing_dealt.csv")

In [31]:
remove_esc_chars(df.iloc[0]['text'])

'About abdominal aortic aneurysms An abdominal aortic aneurysm (AAA) is a swelling (aneurysm) of the aorta the main blood vessel that leads away from the heart, down through the abdomen to the rest of the body. The abdominal aorta is the largest blood vessel in the body and is usually around 2cm wide roughly the width of a garden hose. However, it can swell to over 5.5cm what doctors class as a large AAA. Large aneurysms are rare, but can be very serious. If a large aneurysm bursts, it causes huge internal bleeding and is usually fatal. The bulging occurs when the wall of the aorta weakens. Although what causes this weakness is unclear, smoking and high blood pressure are thought to increase the risk of an aneurysm. AAAs are most common in men aged over 65. A rupture accounts for more than 1 in 50 of all deaths in this group. This is why all men are invited for a screening test when they turn 65. The test involves a simple ultrasound scan, which takes around 10-15 minutes. Symptoms of 

In [32]:
driver.quit()